In [1]:
!pip install transformers
!pip install konlpy
!pip install unidic-lite
!pip install mecab-python3
!sudo apt-get install -y curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 27.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=1caacf757f36b0443d62196632f2dba817144920ff4f9eaaec38afe964033633
  Stored in direc

In [2]:
import transformers
transformers.__version__

'4.29.1'

In [3]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

import re
import nltk
import konlpy
from konlpy.tag import Mecab
mecab = Mecab()

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f87cee67be0>)

In [5]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [6]:
print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력

훈련용 리뷰 개수 : 150000


In [7]:
print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력

테스트용 리뷰 개수 : 50000


In [8]:
train_data.head() # 상위 5개 출력

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [9]:
test_data.head() # 상위 5개 출력

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [10]:
def normalize_text(text):
    if not isinstance(text, str):
        return text
    #HTML 태그 제거
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)

    #이메일 제거
    pattern = r'\S+@\S+'
    text = re.sub(pattern, '', text)

    #전화번호 제거
    pattern = r'\d{2,3}-\d{3,4}-\d{4}'
    text = re.sub(pattern, '', text)

    #링크 제거
    pattern = r'http\S+|www\S+'
    text = re.sub(pattern, '', text)

    # 영어 텍스트 처리
    # 소문자로 변환
    text = text.lower()

    #특수문자 제거
    text = re.sub('[^a-zA-Z0-9가-힣\s]', '', text)

    # 불용어 제거
    stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if not word in stopwords]
    text = ' '.join(tokens)
    
    # 형태소 분석기를 사용한 토큰화
    tokens = mecab.morphs(text)
    
    # 불용어 제거
    stopwords = ['에', '는', '은', '을', '를', '이', '가', '으로', '로', '에서', '과', '와', '의', '도', '다', '만', '들', '은데', '고', '하다', '다']
    tokens = [word for word in tokens if not word in stopwords]
    # 중복된 단어를 하나로 줄이기
    new_tokens = []
    for word in tokens:
        if len(new_tokens) == 0 or word != new_tokens[-1]:
            new_tokens.append(word)
    text = ' '.join(new_tokens)
    
    return text

In [11]:
train_data['document'] = train_data['document'].apply(normalize_text)
test_data['document'] = test_data['document'].apply(normalize_text)

In [12]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
train_data = train_data[train_data['document'] != '']
train_data = train_data.reset_index(drop=True)
print(train_data.isna().sum()) # Null 값이 존재하는지 확인

id          0
document    0
label       0
dtype: int64


In [13]:
test_data = test_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
test_data = test_data[test_data['document'] != '']
test_data = test_data.reset_index(drop=True)
print(test_data.isna().sum()) # Null 값이 존재하는지 확인

id          0
document    0
label       0
dtype: int64


In [14]:
print(len(train_data))

149335


In [15]:
print(len(test_data))

49763


In [16]:
lengths = [len(x) for x in train_data['document']]
sorted_data = train_data.iloc[np.argsort(lengths)][::-1]
sorted_data

,id,document,label
110245,9352925,어릴 적 영화 티비 방송 되 엄마 보 라고 하 시 며 녹화 해 주 시 이후 녹화 한...,1
100381,9097019,140 맞추 기 드럽 게 힘드 네 진짜 이런 쓰레기 세상 에다 있 네 분리수거 해야...,0
57480,8395425,내 노래 잘 알 거나 그런 건 아닌데 솔직히 합 채 랑 썸 띵 중 썸 띵 떨어진 건...,0
83253,8517243,아니 왜 썸 띵 미워하 냐고요 동영상 보 니깐 김아현 눈 밑 다크 써 클 내려와 잇...,0
126001,7928789,눈뜨 본 생방 중 가장 최악 었 던 프로그램 시청 하 너무 불쾌 해 평점 남 깁니다...,0
...,...,...,...
49962,6645285,쩜,1
14217,7247064,애,0
136961,4814547,좋,1
49964,5158262,임,1


In [17]:
'''
Hugging Face에서 제공하는 BERT 모델 중 KLUE(Brain Korea Language Understanding Evaluation)에서 제작한 한국어 BERT 모델 중 하나인 'klue/bert-base'를 사용
'''
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [18]:
print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

[2, 1160, 2259, 2369, 2369, 4311, 20657, 2259, 5501, 13132, 1415, 2259, 23713, 3]


In [19]:
print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

['보', '##는', '##내', '##내', '그대로', '들어맞', '##는', '예측', '카리스마', '없', '##는', '악역']


In [20]:
tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

'[CLS] 보는내내 그대로 들어맞는 예측 카리스마 없는 악역 [SEP]'

In [21]:
for elem in tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"):
  print(tokenizer.decode(elem))

[ C L S ]
보
# # 는
# # 내
# # 내
그 대 로
들 어 맞
# # 는
예 측
카 리 스 마
없
# # 는
악 역
[ S E P ]


In [22]:
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)
print(tokenizer.pad_token, ':', tokenizer.pad_token_id)

[CLS] : 2
[SEP] : 3
[PAD] : 0


In [23]:
train_data['document'].apply(len).max()

190

In [24]:
max_seq_len = 200

In [25]:
encoded_result = tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역", max_length=max_seq_len, pad_to_max_length=True)
print(encoded_result)
print('길이 :', len(encoded_result))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[2, 1160, 2259, 2369, 2369, 4311, 20657, 2259, 5501, 13132, 1415, 2259, 23713, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
길이 : 200


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [26]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True)
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [27]:
train_X, train_y = convert_examples_to_features(train_data['document'], train_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 149335/149335 [01:04<00:00, 2303.46it/s]


In [28]:
test_X, test_y = convert_examples_to_features(test_data['document'], test_data['label'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 49763/49763 [00:18<00:00, 2732.94it/s]


In [29]:
input_id = train_X[0][0]
attention_mask = train_X[1][0]
token_type_id = train_X[2][0]
label = train_y[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))
print('레이블 :',label)

단어에 대한 정수 인코딩 : [   2 1376  831 1193 4229 9801  717 4363 4243    3    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0 

In [30]:
model = TFBertModel.from_pretrained("klue/bert-base", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'bert.embeddings.position_ids', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [31]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
attention_masks_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
token_type_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)

outputs = model([input_ids_layer, attention_masks_layer, token_type_ids_layer])

In [32]:
print(outputs)

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 200, 768) dtype=float32 (created by layer 'tf_bert_model')>, pooler_output=<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>, past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)


In [33]:
print(outputs[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 200, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model'")


In [34]:
print(outputs[1])

KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='tf_bert_model/bert/pooler/dense/Tanh:0', description="created by layer 'tf_bert_model'")


In [35]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(1,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='sigmoid',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [36]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [37]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [38]:
with strategy.scope():
  model = TFBertForSequenceClassification("klue/bert-base")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'bert.embeddings.position_ids', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [39]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=3)

checkpoint_path = 'weights.h5'

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

In [40]:
model.fit(train_X, train_y, epochs=10, batch_size=64, validation_split=0.2,callbacks=[earlystop_callback, cp_callback])

Epoch 1/10
1867/1867 [==============================] - ETA: 0s - loss: 0.3072 - accuracy: 0.8678
Epoch 1: val_accuracy improved from -inf to 0.88978, saving model to weights.h5
1867/1867 [==============================] - 424s 174ms/step - loss: 0.3072 - accuracy: 0.8678 - val_loss: 0.2622 - val_accuracy: 0.8898
Epoch 2/10
1867/1867 [==============================] - ETA: 0s - loss: 0.2126 - accuracy: 0.9151
Epoch 2: val_accuracy did not improve from 0.88978
1867/1867 [==============================] - 263s 141ms/step - loss: 0.2126 - accuracy: 0.9151 - val_loss: 0.2807 - val_accuracy: 0.8873
Epoch 3/10
1867/1867 [==============================] - ETA: 0s - loss: 0.1459 - accuracy: 0.9450
Epoch 3: val_accuracy improved from 0.88978 to 0.89028, saving model to weights.h5
1867/1867 [==============================] - 269s 144ms/step - loss: 0.1459 - accuracy: 0.9450 - val_loss: 0.3062 - val_accuracy: 0.8903
Epoch 4/10
1867/1867 [==============================] - ETA: 0s - loss: 0.0980 - 

In [41]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

49/49 [==============================] - 32s 506ms/step - loss: 0.5673 - accuracy: 0.8867
test loss, test acc:  [0.5672987103462219, 0.8867230415344238]


In [42]:
model.load_weights(checkpoint_path)

# best model을 이용한 테스트 데이터 예측 정확도 재확인 코드
test_loss, test_acc = model.evaluate(test_X, test_y, verbose=1)
test_loss,test_acc

1556/1556 [==============================] - 70s 41ms/step - loss: 0.4755 - accuracy: 0.8903


(0.47551941871643066, 0.8902598023414612)

In [43]:
def sentiment_predict(new_sentence):
  input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

  padding_count = input_id.count(tokenizer.pad_token_id)
  attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
  token_type_id = [0] * max_seq_len

  input_ids = np.array([input_id])
  attention_masks = np.array([attention_mask])
  token_type_ids = np.array([token_type_id])

  encoded_input = [input_ids, attention_masks, token_type_ids]
  score = model.predict(encoded_input)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [60]:
sentiment_predict("이 영화 존잼입니다 대박")

1/1 [==============================] - 1s 1s/step
95.50% 확률로 긍정 리뷰입니다.



In [61]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

1/1 [==============================] - 1s 1s/step
98.51% 확률로 긍정 리뷰입니다.



In [63]:
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')

1/1 [==============================] - 1s 1s/step
98.41% 확률로 긍정 리뷰입니다.



In [64]:
sentiment_predict('제 점수는 10점 만점에 1점입니다')

1/1 [==============================] - 1s 891ms/step
99.82% 확률로 부정 리뷰입니다.



In [65]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

1/1 [==============================] - 1s 962ms/step
99.81% 확률로 부정 리뷰입니다.



In [66]:
sentiment_predict('감독 뭐하는 놈이냐?')

1/1 [==============================] - 1s 903ms/step
99.96% 확률로 부정 리뷰입니다.



In [68]:
sentiment_predict('쪠 인썡 최꾜의 영화잆니땨')

1/1 [==============================] - 2s 2s/step
78.92% 확률로 부정 리뷰입니다.



In [78]:
sentiment_predict('영홙갅 잱믽는걵 같앉욙 춙천 듡립닍닩')

1/1 [==============================] - 2s 2s/step
78.19% 확률로 부정 리뷰입니다.



In [70]:
sentiment_predict('쓰툐리뚀 탼탼햐꾜 뺴유뜰의 연끼렦뚀 쭇야요')

1/1 [==============================] - 2s 2s/step
78.19% 확률로 부정 리뷰입니다.



In [81]:
sentiment_predict('영화 스토리가 ㅈㄴ미쳤어요')

1/1 [==============================] - 4s 4s/step
99.01% 확률로 부정 리뷰입니다.



In [80]:
sentiment_predict('배우들 연기 ㅈㄴ지림')

1/1 [==============================] - 2s 2s/step
99.59% 확률로 부정 리뷰입니다.



In [85]:
sentiment_predict('영화가 그냥 ㅈㄴ 미쳤어요')

1/1 [==============================] - 1s 923ms/step
51.31% 확률로 긍정 리뷰입니다.

